# Essential tools to write better Python

**Overview**
 * List comprehensions
 * Decorators
 * Good documentation 

## List comprehensions

List comprehensions provide a compact and readible way to create lists. 


**Syntax**:

Create a list without list comprehension:

```python
new_list = []
for i in old_list:
    if filter(i):
        new_list.append(my_func(i))
```        
the same task with list comprehension

```python
new_list = [my_func(i) for i in old_list if filter(i)]
```

### Example 1: List of even numbers

**Task**: Create a list of even numbers.

**Solution** without list comprehension:

In [57]:
def is_even(i):
    return i%2==0

even_numbers = []
for i in range(20):
    if is_even(i):
        even_numbers.append(i)
print(even_numbers)

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]


**Solution** with list comprehension:

In [58]:
even_numbers = [i for i in range(20) if i%2==0]
even_numbers

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [59]:
### Example 2: Remove sensitive information from log data

**Task**: Remove all strings in a logfile that contain passwords

**Solution** without list comprehension:

SyntaxError: invalid syntax (<ipython-input-59-6c77a565936a>, line 3)

In [70]:
fp = open("log.txt", "r")

log = []
for line in fp:
    if "password" not in line:
        log.append(line.strip())
fp.close() 

log

['09.Sept 2019 14:30: New user enters webpage',
 '09.Sept 2019 14:31: Login email: simon@simula.no',
 '09.Sept 2019 14:35: User leaves webpage']

**Solution** with list comprehension:

In [71]:
with open('log.txt', "r") as fp:
    log = [line.strip() for line in fp if "password" not in line]
    
log    

['09.Sept 2019 14:30: New user enters webpage',
 '09.Sept 2019 14:31: Login email: simon@simula.no',
 '09.Sept 2019 14:35: User leaves webpage']

## Decorators  in Python

In [36]:
def uppercase(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs).upper()
    return wrapper

def lowercase(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs).lower()
    return wrapper

def debug(func):
    def wrapper(*args, **kwargs):
        print(f"DEBUG: Calling function:   {str(func)})")
        print(f"DEBUG: Function arguments: {args}, keyword arguments {kwargs})")
        out = func(*args, **kwargs).lower()
        print(f"DEBUG: function output: '{out}', (type {type(out)})")
        return out
    return wrapper

## What is a decorator?

A decorator 
**allows a user to add new functionality to an existing object without modifying its structure**. 

Demo:

In [39]:
#@upper  # <- Specify a decorator name here
def greet(name):
    return f"Hello world, {name}"
    
    
greet(name="Simon")

'Hello world, Simon'

## Functions as arguments

Like all objects, functions can be arguments to functions

In [12]:
def add(x,y):
    return x+y

def sub(x, y):
    return x-y

def apply(func, x, y):
    return func(x, y)

In [13]:
apply(add, 1, 2)

3

In [14]:
apply(sub, 7, 5)

2

## Functions inside functions

Python allows nested function definitions:

In [15]:
def g(x, y):
    
    def cube(x):
        return x*x*x
    
    return y*cube(x)

g(4, 6)

384

## Function returning functions

In [21]:
def h():
    pi = 0.13
    def inner_h():
        print("Inside inner_h but can access pi={}".format(pi))
        
    return inner_h

foo = h()
foo

<function __main__.h.<locals>.inner_h>

In [20]:
foo()

Inside inner_h but can access pi='0.13'


## More functions returning functions: *decorators*

A toy example

In [1]:
def foo():
    return 1

def outer(func):
    def inner():
        print("before calling func")
        return func()
    return inner

In [2]:
decorated = outer(foo)

The function `decorated` is a decorated version of function `foo`.
It is `foo` plus something more:

In [3]:
decorated()

before calling func


1

To simplify, we could just write
```python 
foo = outer(foo)
```
to replace foo with its decorated version each time it is called

## A (slightly) more useful decorator

Suppose we have been given a function that only works for some numerical inputs:

In [5]:
from math import log
def f(x):
    return log(x) - 2

In [6]:
f(5)

-0.3905620875658997

In [7]:
f(-1)

ValueError: math domain error

Suppose we want to limit the range of values sent to this function:

The idea is that we **wrap** the function inside another function:

In [28]:
def checkrange(func):
    def inner(x, limit):
        if x <= limit:
            print("out of range")
        else:
            return func(x)
    return inner

In [32]:
fcheck = checkrange(f)
fcheck(5, limit=6)

out of range


In [10]:
fcheck(-1)

out of range


Voilà!!

## The `@decorator` syntax

Python provides a short notation for decorating a function with
another function:

In [15]:
@checkrange
def g(x):
    return log(x) - 2

In [16]:
g(0)

out of range


This is essentially the same as writing `g = checkrange(g)`.

A decorator is simply a function taking another function as input
and returning another function. 

The syntax `@decorator` is a
short-cut for the more explicit `f = decorator(f)`.

## A (much) more useful decorator: memoization

Assume we have a slow function. Something like

In [17]:
from time import sleep

def slow(x, y):
    res = x*y; sleep(1)     # Simulate a long computation
    return res

In [18]:
%timeit slow(1, 2)

1 s ± 145 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


We call the function with the same input arguments, and hence perform the same (slow) calculations multiple times.

The idea of memoization (or buffering) is to buffer the input-output pairs for which the function was called.
If the function is called twice with same input arguments, we return the buffer value.

The implementation of a memoization with a `decorator` could look like:

In [20]:
cache = {}  # Stores all input-output pairs

def memoize(func):
    ''' Caches a function's return value each time it is called.
        If called later with the same arguments, the cached value is returned
        (not reevaluated). '''
    
    def inner(x, y):
        if (x, y) in cache:
            return cache[(x, y)]
        else:
            result = func(x, y)
            cache[(x, y)] = result
            return result
        
    return inner

Now we can apply the decorator to our slow function

In [21]:
@memoize
def slow(x, y):
    res = x*y; sleep(1)     # Simulate a long computation
    return res

... and test it out

In [27]:
slow(1, 2)

2

## Decorator summary 

* A function that takes a function as argument and returns a modified function
* `@decorator` syntax simply a short cut for the standard function call `f = decorator(f)`.